In [ ]:
!pip install tensorflow==2.9.1
!pip install transformers==4.37.2
#!pip install -q -U keras-tuner
!pip3 install --upgrade mlflow

### get text and image models

In [ ]:
root_model_text = []
models_runs = ['50c85a2deab242f1ad4085341ef08867', '6ee6adbd7e854df7bf6489386fe374f7', 'f97b6bc180cf44d98dd3df6c5261588d', '642a6cff6b534df1b64b2c37bda832e5', 'afa78a883cef4cb9957f85835b07698d']
artifacts_dir = '/content/drive/MyDrive/final_project/bert/multiple_inputs/small_model_joined_data/models_compartida/artifacts/' + models_runs[LEVEL -1] + '/artifacts/'

config_root = artifacts_dir + 'model_config.json'
weights_root = artifacts_dir + 'model_weights.h5'

with open(config_root, 'r') as file_json:
  config = json.load(file_json)
model = tf.keras.models.model_from_json(json.dumps(config))
string_name = f'model_text_L{LEVEL}'

globals()[string_name] = model
globals()[string_name].load_weights(weights_root)
globals()[string_name]._name = string_name
globals()[string_name].summary()

In [ ]:
root_model_image = []
artifacts_dir_image = '/content/drive/MyDrive/final_project/bert/multiple_inputs/small_model_joined_data/models_compartida/models/images/resnet_0_64/'

config_root = artifacts_dir_image + 'RESENET_60_model_config.json'
weights_root_image = artifacts_dir_image + 'RESENET_60_model_weights.h5'

with open('/content/drive/MyDrive/final_project/bert/multiple_inputs/small_model_joined_data/models_compartida/models/images/resnet/cnn_11_3ago_config.pkl', 'rb') as file_pkl:
  config = pickle.load(file_pkl)
with open('/content/drive/MyDrive/final_project/bert/multiple_inputs/small_model_joined_data/models_compartida/models/images/resnet/cnn_11_3ago_weights.pkl', 'rb') as file_pkl:
  weights = pickle.load(file_pkl)
model = tf.keras.Model.from_config(config)
string_name = f'model_image_L{LEVEL}'
globals()[string_name] = model
globals()[string_name].set_weights(weights)
globals()[string_name]._name = string_name
globals()[string_name].summary()

In [ ]:
## set traible = False to both models
models_L1 = [model_text_L1, model_image_L1, ]
for model in models_L1:
  model.trainable = False

### Build the model

In [ ]:
###
input_text = Input((768, ), name='input_text')
input_image = Input((224, 224, 3,), name='input_image')

out_text = tf.nn.softmax(models_L1[0](input_text))
out_image = models_L1[1](input_image)

x_c = concatenate([out_text, out_image])
out_layer = Dense(13,)(x_c)

model = tf.keras.Model([input_text, input_image], out_layer)
model.compile('adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', F1Score(13, average='weighted')])

### Settiing up the experiment tracking

In [ ]:
# setting up mlflow
shared_folder = '/content/drive/MyDrive/final_project/bert/multiple_inputs/small_model_joined_data/models_compartida'
store = '/my_runs.db'
mlflow.set_tracking_uri('sqlite:///'+shared_folder + store)
print(mlflow.get_tracking_uri())

In [ ]:
def create_experiment():
  """
    This function creates the experiment in the experiment tracking database

    Return:
      LEVEL: int level of the category
      MLFLOW_EXPERIMENT_NAME: str name of the experiment
  """
  global LEVEL
  global EXPERIMENT_NAME

  EXPERIMENT_NAME = f'fusion model level {LEVEL}'
  artifact_folder = 'artifacts'
  artifact_location = os.path.join(shared_folder, artifact_folder)

  try:
    mlflow_experiment_id = mlflow.create_experiment(EXPERIMENT_NAME,
                                                    artifact_location=artifact_location)
  except Exception as e:
    print(e)
  finally:
    print('set experiment')
    mlflow.set_experiment(EXPERIMENT_NAME)


### Setting up graphs and metrics

In [ ]:
class F1Score(tf.keras.metrics.Metric):
  """
    This function is a customized matric to evaluate the F1 score in the training
  """
  def __init__(self, num_classes, average='macro', name='f1_score', **kwargs):
    super(F1Score, self).__init__(name=name, **kwargs)
    self.num_classes = num_classes
    self.average = average
    self.true_positives = self.add_weight(name='tp', shape=(num_classes,), initializer='zeros', dtype=tf.float32)
    self.false_positives = self.add_weight(name='fp', shape=(num_classes,), initializer='zeros', dtype=tf.float32)
    self.false_negatives = self.add_weight(name='fn', shape=(num_classes,), initializer='zeros', dtype=tf.float32)

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_pred = tf.argmax(y_pred, axis=-1)
    y_true = tf.argmax(y_true, axis=-1)

    for i in range(self.num_classes):
      y_pred_i = tf.cast(y_pred == i, tf.float32)
      y_true_i = tf.cast(y_true == i, tf.float32)

      tp = tf.reduce_sum(y_true_i * y_pred_i)
      fp = tf.reduce_sum(y_pred_i) - tp
      fn = tf.reduce_sum(y_true_i) - tp

      self.true_positives[i].assign(self.true_positives[i] + tp)
      self.false_positives[i].assign(self.false_positives[i] + fp)
      self.false_negatives[i].assign(self.false_negatives[i] + fn)

  def result(self):
    precision = self.true_positives / (self.true_positives + self.false_positives + K.epsilon())
    recall = self.true_positives / (self.true_positives + self.false_negatives + K.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())

    if self.average == 'macro':
      return tf.reduce_mean(f1)
    elif self.average == 'weighted':
      weights = self.true_positives + self.false_negatives
      return tf.reduce_sum(f1 * weights) / tf.reduce_sum(weights)
    else:
      raise ValueError(f'Unknown average type: {self.average}')

  def reset_state(self):
    for i in range(self.num_classes):
      self.true_positives[i].assign(0)
      self.false_positives[i].assign(0)
      self.false_negatives[i].assign(0)


In [ ]:
def get_model_plot(model):
  """
    This function creates the model architecture graph.
  """
  tf.keras.utils.plot_model(model, '/content/plot_model.png',
                                        show_shapes=True,
                                        show_dtype=True,
                                        show_layer_names=True,
                                        show_layer_activations=True,
                                        rankdir='PR'
                                        )
def plots_gra(history, save=None):
  """
    This function creates the losses and accuracy plots
  """
  metrics = ['loss', 'accuracy']

  fig, axis = plt.subplots(2,2, figsize=(10, 8))

  axis[0, 0].plot(history.history['loss'])
  axis[0, 0].set_xlabel('epochs')
  axis[0, 0].set_ylabel('loss')

  axis[0, 1].plot(history.history[f'val_loss'])
  axis[0, 1].set_xlabel('epochs')
  axis[0, 1].set_ylabel(f'val_loss')

  axis[1, 0].plot(history.history['accuracy'])
  axis[1, 0].set_xlabel('epochs')
  axis[1, 0].set_ylabel('accuracy')

  axis[1, 1].plot(history.history[f'val_accuracy'])
  axis[1, 1].set_xlabel('epochs')
  axis[1, 1].set_ylabel(f'val_accuracy')

  fig.tight_layout()

  if save:
    plt.savefig('/content/metrics.png')
    #fig.savefig(save_root + '/metrics.png')
    #plt.close()
  plt.show()

def get_consufion_matrix(set_to_evaluate, true_labels, name):

  """
      this function is to create the confusion matrix with my results
      inputs:
        set_to_evaluate = set of embeddings to evaluate
        true_labels = the labels of set_to_evaluate
        root = Path in where it confussion matrix will be save
  """

  global class_names

  predict = model.predict(set_to_evaluate)
  predict = np.argmax(predict, axis=1)

  cm = confusion_matrix(true_labels, predict)

  # Crear una figura y un eje
  fig, ax = plt.subplots(figsize=(10, 8))

  # Crear el mapa de calor de la matriz de confusión
  sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, ax=ax)

  # Añadir etiquetas y título
  ax.set_xlabel('Predicted labels')
  ax.set_ylabel('True labels')
  ax.set_title('Confusion Matrix')

  fig.tight_layout()
  if name:
    #fig.savefig('/content/drive/MyDrive/final_project/bert/multiple_inputs/models/0.9330/con_matrix.png')
    #print('guardo')
    fig.savefig(f'/content/con_matrix_{name}.png')

In [ ]:
### setting callbacks and clear enviroment functions
# Funtion to clear my dir
def clear_enviroment():
  tf.keras.backend.clear_session()
  if 'model' in dir() :
    print("i am cutting")
    del(model)

# Callbacks
call_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=9, restore_best_weights=True)
class CustomCallback(tf.keras.callbacks.Callback):
  """
    Customizer class to log the metrics model in mlflow
  """
  def on_epoch_end(self, epoch, logs=None):
    """
      This function log the metrics on epoch end
    """
    mlflow.log_metric('loss', logs.get('loss'), step=epoch)
    mlflow.log_metric('accuracy', logs.get('accuracy'), step=epoch)
    mlflow.log_metric('val_loss', logs.get('loss'), step=epoch)
    mlflow.log_metric('val_accuracy', logs.get('val_accuracy'), step=epoch)

    ## f1_score
    mlflow.log_metric('f1_score', logs.get('f1_score'), step=epoch)
    mlflow.log_metric('val_f1_score', logs.get('val_f1_score'), step=epoch)
    """
    mlflow.log_metric("accuracy", logs["accuracy"])
    mlflow.log_metric("loss", logs["loss"])

### Obtain the dato for level

In [ ]:
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.03),
])

def load_image(file_name):
  raw = tf.io.read_file(file_name)
  tensor = tf.image.decode_jpeg(raw, channels=3)
  tensor2 = tf.image.resize(tensor, [224, 224], preserve_aspect_ratio=True)
  shape = tf.shape(tensor2)
  h = (224 - shape[0]) //2
  w = (224 - shape[1]) //2
  h = tf.maximum(h, 0)
  w = tf.maximum(w, 0)
  tensor = tf.image.pad_to_bounding_box(tensor2, int(h.numpy()), int(w.numpy()), 224, 224)
  mask = tf.image.pad_to_bounding_box(tf.ones_like(tensor2), h, w, 224, 224)
  tensor = tf.cast(tensor, tf.float32) / 255.0
  tensor = tensor * mask + (1 - mask) * 0.5
  tensor = data_augmentation(tensor)
  return tensor


def get_mask_of_filtered(set_no, set_fil):
  list_index = []
  m = 0
  for n in range(set_no.shape[0]):
    if set_no.iloc[n,0] == set_fil.iloc[m,0]:
      list_index.append(n)
      m += 1

  list_fil = [False]*set_no.shape[0]
  ran_list_fil = range(set_no.shape[0])

  for ran in ran_list_fil:
    if ran in list_index:
      list_fil[ran] = True

  return list_fil


def load_data():
  """
    This function obtains the data, like data.csv and embeddings from bert.

    Return:
      train: Dataframe.
      test: Daraframe.
      val: Dataframe.
      train_data_join: np.array.
      test_data_join: np.array.
      val_data_join: np.array.
  """

  global train
  global test
  global val
  global train_data_join
  global test_data_join
  global val_data_join

  print('loaded data')

  train = pd.read_csv('/content/drive/MyDrive/final_project/bert/dataset/train.csv')
  test = pd.read_csv('/content/drive/MyDrive/final_project/bert/dataset/test.csv')
  val = pd.read_csv('/content/drive/MyDrive/final_project/bert/dataset/val.csv')

  train_filtered = pd.read_csv('/content/drive/MyDrive/final_project/bert/multiple_inputs/small_model_joined_data/models_compartida/Final/imagenes/train_filtered.csv')
  test_filtered = pd.read_csv('/content/drive/MyDrive/final_project/bert/multiple_inputs/small_model_joined_data/models_compartida/Final/imagenes/test_filtered.csv')
  val_filtered = pd.read_csv('/content/drive/MyDrive/final_project/bert/multiple_inputs/small_model_joined_data/models_compartida/Final/imagenes/val_filtered.csv')

  mask = get_mask_of_filtered(train, train_filtered)
  train = train[mask]
  with open("/content/drive/MyDrive/final_project/bert/dataset/embeddings_joined/train_embeddings.pkl", "rb") as f:
    train_data_join = pickle.load(f)
    train_data_join = np.array(train_data_join)[mask]

  mask = get_mask_of_filtered(test, test_filtered)
  test = test[mask]
  with open("/content/drive/MyDrive/final_project/bert/dataset/embeddings_joined/test_embeddings.pkl", "rb") as f:
    test_data_join = pickle.load(f)
    test_data_join = np.array(test_data_join)[mask]

  mask = get_mask_of_filtered(val, val_filtered)
  val = val[mask]
  with open("/content/drive/MyDrive/final_project/bert/dataset/embeddings_joined/val_embeddings.pkl", "rb") as f:
    val_data_join = pickle.load(f)
    val_data_join = np.array(val_data_join)[mask]

  del(train_filtered)
  del(test_filtered)
  del(val_filtered)

def get_data():
   """
    This function creates the Datasets to train and validate the model.

    Return:
      dataset_train: tf.data.Dataset.
      dataset_test: tf.data.Dataset.
  """
  global LEVEL
  global NUMBER_OUT
  global COMPUTE_CLASS
  global train_label_le
  global train_label_cate
  global test_label_le
  global test_label_cate
  global val_label_le
  global val_label_cate
  global dataset_train
  global dataset_test
  global train
  global test
  global val
  global train_data_join
  global test_data_join
  global val_data_join
  global class_names

  load_data()

  artifacts = '/content/drive/MyDrive/final_project/bert/multiple_inputs/small_model_joined_data/models_compartida/artifacts/Label_encoders'
  ima_folder = '/content/drive/MyDrive/final_project/bert/multiple_inputs/small_model_joined_data/models_compartida/Final/imagenes/img/product_images/'

  train_labels = train[f'subcat{LEVEL}_name']
  test_labels = test[f'subcat{LEVEL}_name']
  val_labels = val[f'subcat{LEVEL}_name']

  train_images_names = np.array(train['image'])
  test_images_names = np.array(test['image'])
  val_images_names = np.array(val['image'])

  if 1 == LEVEL:
    train_labels[pd.isna(train_labels)] = 'Other'
    test_labels[pd.isna(test_labels)] = 'Other'
    val_labels[pd.isna(val_labels)] = 'Other'
  else:
    mask_null_train = ~pd.isna(train_labels)
    mask_null_test = ~pd.isna(test_labels)
    mask_null_val = ~pd.isna(val_labels)

    train_labels = train_labels[mask_null_train]
    test_labels = test_labels[mask_null_test]
    val_labels = val_labels[mask_null_val]

    train_data_join = train_data_join[mask_null_train]
    test_data_join = test_data_join[mask_null_test]
    val_data_join = val_data_join[mask_null_val]

    train_images_names = ima_folder + train_images_names[mask_null_train]
    test_images_names = ima_folder + test_images_names[mask_null_test]
    val_images_names = ima_folder + val_images_names[mask_null_val]


  listdis = os.listdir(artifacts)

  if f'label_encoder_level{LEVEL}.pkl' in listdis:
    with open(f'{artifacts}/label_encoder_level{LEVEL}.pkl', 'rb') as f:
      le = pickle.load(f)
      train_label_le = le.transform(train_labels)
    print('loaded')
  else:
    le = LabelEncoder()
    train_label_le = le.fit_transform(train_labels)
    with open(f'{artifacts}/label_encoder_level{LEVEL}.pkl', 'wb') as f:
      pickle.dump(le, f)
    print('created and saved')

  class_names = le.classes_
  train_label_cate = to_categorical(train_label_le)
  test_label_le = le.transform(test_labels)
  test_label_cate = to_categorical(test_label_le)
  val_label_le = le.transform(val_labels)
  val_label_cate = to_categorical(val_label_le)
  NUMBER_OUT = len(train_labels.unique())
  print(len(train_label_cate))
  print(len(test_label_cate))
  print(NUMBER_OUT)

  COMPUTE_CLASS = compute_class_weight(class_weight="balanced", classes=np.unique(train_label_le), y=train_label_le)
  COMPUTE_CLASS = {i:value for i, value in enumerate(COMPUTE_CLASS) }
  print(COMPUTE_CLASS)

  print(train_images_names.shape, train_data_join.shape)

  dataset_train = tf.data.Dataset.from_tensor_slices(((train_data_join, train_images_names), train_label_cate))
  dataset_train = dataset_train.map(lambda x, y: ((x[0], (load_image(ima_folder + x[1]))), y), num_parallel_calls=tf.data.AUTOTUNE)
  dataset_train = dataset_train.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  dataset_test = tf.data.Dataset.from_tensor_slices(((val_data_join, val_images_names), val_label_cate))
  dataset_test = dataset_test.map(lambda x, y: ((x[0], (load_image(ima_folder + x[1]))), y), num_parallel_calls=tf.data.AUTOTUNE)
  dataset_test = dataset_test.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

### Experiment tracking

In [ ]:
EXPERIMENT_NUMBER = 1
EXPERIMENT_NAME = 'Fusion Models'

In [ ]:
LEVEL = 1
create_experiment()
get_data()

In [ ]:
THRESHOLD = 0.70 ### threshold to exceed to save the model
EPOCHS = 1000
with open('/content/drive/MyDrive/final_project/bert/multiple_inputs/small_model_joined_data/models_compartida/my_runs.db'):

  mlflow.start_run(run_name=f'{EXPERIMENT_NAME}_N_{EXPERIMENT_NUMBER}')
  clear_enviroment()

  histoty = model.fit(dataset_train, epochs=EPOCHS, validation_data=dataset_test,  class_weight=COMPUTE_CLASS, callbacks=[call_early, CustomCallback()])

  ### Obtain the graphs and metrics
  get_model_plot(model, )
  plots_gra(history, True)
  #test
  get_consufion_matrix(test_data_join, test_label_le, 'test')
  #val
  get_consufion_matrix(val_data_join, val_label_le, 'val')

  if 'metrics.png' in os.listdir('/content'):
    os.rename('/content/metrics.png', '/content/old_metrics.png')
    print('this is the old metric')
    fig = plt.figure(figsize=(10, 10))
    old_img = tf.keras.preprocessing.image.load_img('/content/old_metrics.png')
    plt.imshow(old_img)

  ### Obtain the predictions for validation and test sets
  predict_test = model.predict(test_data_join)
  predict_test = np.argmax(predict_test, axis=1)
  f1_test = f1_score(test_label_le, predict_test, average='weighted')
  print(f'{f1_test=}')

  predict_val = model.predict(val_data_join)
  predict_val = np.argmax(predict_val, axis=1)
  f1_val = f1_score(val_label_le, predict_val, average='weighted')
  print(f'val = {f1_val=}')

  if f1_val >= THRESHOLD and f1_test >= THRESHOLD :
    print('the threshold has been overcome')
    config = model.get_config()
    weights = model.get_weights()
    model.save('/content/model.keras')
    model.save_weights('model_weights.h5')
    model_config = model.to_json()

    with open('model_config.json', 'w') as json_file:
      json_file.write(model_config)

    mlflow.log_artifact('model_weights.h5',)
    mlflow.log_artifact('model_config.json',)
    mlflow.log_artifact('model.keras')

EXPERIMENT_NUMBER += 1
